In [63]:
import time
import torch
from torch import nn, optim
import torchvision
import math
from LeNet import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LENet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)
training on  cuda
epoch 1, loss 1.7855, train acc 0.351, test acc 0.596, time 11.8 sec
epoch 2, loss 0.4620, train acc 0.647, test acc 0.686, time 7.0 sec
epoch 3, loss 0.2480, train acc 0.726, test acc 0.730, time 7.3 sec
epoch 4, loss 0.1655, train acc 0.747, test acc 0.747, time 7.3 sec
epoch 5, loss 0.1224, train acc 0.762, test acc 0.762, time 7.5 sec
epoch 6, loss 0.0962, train acc 0.773, t

In [50]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4),#(input_channel, output_channel, kernal_size, stride, padding)
            nn.ReLU(),
            nn.MaxPool2d(3, 2),#(kernal_size, stride)
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )
    
    def forward(self, img):
        feature = self.conv(img)
#        print(feature.size())
        return self.fc(feature.view(img.shape[0], -1))
    

net = AlexNet()
print(net)

#print(net.parameters)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (

In [51]:
def compute_map_size(size):
    param = []
    for layer in net.modules():
        if isinstance(layer, nn.Conv2d):
            param.append([layer.kernel_size[0],layer.stride[0],layer.padding[0]])
            print("kernal_size: %d, stride: %d, padding: %d" % (layer.kernel_size[0],layer.stride[0],layer.padding[0]))
        if isinstance(layer, nn.MaxPool2d):
            param.append([layer.kernel_size,layer.stride,layer.padding])
            print("kernal_size: %d, stride: %d, padding: %d" % (layer.kernel_size,layer.stride,layer.padding))
    print(param)
    cur_size = size
    for k, s, p in param:
        cur_size = (cur_size - k + 2*p + s) // s
        size = math.floor((size + 2*p -k) / s) + 1
    print(cur_size, size)
compute_map_size(224)

kernal_size: 11, stride: 4, padding: 0
kernal_size: 3, stride: 2, padding: 0
kernal_size: 5, stride: 1, padding: 2
kernal_size: 3, stride: 2, padding: 0
kernal_size: 3, stride: 1, padding: 1
kernal_size: 3, stride: 1, padding: 1
kernal_size: 3, stride: 1, padding: 1
kernal_size: 3, stride: 2, padding: 0
[[11, 4, 0], [3, 2, 0], [5, 1, 2], [3, 2, 0], [3, 1, 1], [3, 1, 1], [3, 1, 1], [3, 2, 0]]
5 5


In [56]:
def load_data_fashion_mnist(batch_size, resize=None, root='D:\Repos\PycharmProjects\Datasets\FashionMNIST'):
    # torchvision.transform包含一些图像变换方法
    # 图像预处理
    trans=[]
    # 使用torchvision.transform.Resize(size)来实现图像大小变化
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    # 图像格式转换成tensor
    trans.append(torchvision.transforms.ToTensor()) 
    # 将所有transform操作合并
    transform = torchvision.transforms.Compose(trans)
    
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True,transform=transform)
    
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)
    
    return train_iter, test_iter

In [57]:
batch_size = 128
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [66]:
lr = 0.001
num_epochs = 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()

train(net, train_iter, test_iter, optimizer, loss, device, num_epochs)

training on  cuda
epoch 1, loss 0.4096, train acc 0.848, test acc 0.841, time 7.5 sec
epoch 2, loss 0.2001, train acc 0.853, test acc 0.840, time 7.3 sec
epoch 3, loss 0.1307, train acc 0.856, test acc 0.844, time 7.2 sec
epoch 4, loss 0.0962, train acc 0.858, test acc 0.844, time 7.1 sec
epoch 5, loss 0.0757, train acc 0.861, test acc 0.842, time 7.1 sec
epoch 6, loss 0.0616, train acc 0.866, test acc 0.850, time 7.3 sec
epoch 7, loss 0.0522, train acc 0.867, test acc 0.851, time 7.0 sec
epoch 8, loss 0.0447, train acc 0.869, test acc 0.856, time 7.1 sec
epoch 9, loss 0.0393, train acc 0.870, test acc 0.857, time 7.3 sec
epoch 10, loss 0.0348, train acc 0.872, test acc 0.859, time 7.1 sec
